# 使用 AudioEncoder 对音频样本进行编码

先生成一些样本进行编码。需要编码的数据也可以直接从[AudioEncoder](https://meta-pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder)获取！

In [1]:
import torch
from IPython.display import Audio as play_audio


def make_sinewave() -> tuple[torch.Tensor, int]:
    """生成 440Hz （音乐中的标准音高 A4，即中央 C 上方的 A 音）的正弦波音频，时长 3 秒 ，采样率为 16000Hz。
    """
    freq_A = 440  # Hz (频率：决定音高)
    sample_rate = 16000  # Hz (采样率：每秒采集/播放的数据点数)
    duration_seconds = 3  # seconds (时长：音频的总时间长度)
    t = torch.linspace(0, duration_seconds, int(sample_rate * duration_seconds), dtype=torch.float32)
    return torch.sin(2 * torch.pi * freq_A * t), sample_rate


samples, sample_rate = make_sinewave()

print(f"Encoding samples with {samples.shape = } and {sample_rate = }")
play_audio(samples, rate=sample_rate)  # 音频播放控件


Encoding samples with samples.shape = torch.Size([48000]) and sample_rate = 16000


首先实例化音频编码器 。样本必须是二维张量 （num_channels，num_samples）），或者在这里，是一维张量，其中 num_channels 假设为 1.这些值必须是以 $[-1， 1]$ 归一化的浮点值：这也是 音频解码器会回归。

In [2]:
from torchcodec.encoders import AudioEncoder

encoder = AudioEncoder(samples=samples, sample_rate=sample_rate)

`sample_rate` 参数对应于 输入 ，而不是期望的编码采样率。

`AudioEncoder` 支持通过 `to_file` 方法将采样编码成文件，或通过 `to_tensor` 方法编码为原始字节。


In [3]:
encoded_samples = encoder.to_tensor(format="mp3")
print(f"{encoded_samples.shape = }, {encoded_samples.dtype = }")

encoded_samples.shape = torch.Size([9512]), encoded_samples.dtype = torch.uint8


有了编码数据，可以重新解码，确保它看起来和听起来都符合预期：

In [4]:
from torchcodec.decoders import AudioDecoder

samples_back = AudioDecoder(encoded_samples).get_all_samples()

print(samples_back)
play_audio(samples_back.data, rate=samples_back.sample_rate)

AudioSamples:
  data (shape): torch.Size([1, 48000])
  pts_seconds: 0.0690625
  duration_seconds: 3.0
  sample_rate: 16000



编码器支持一些编码选项，允许你更改数据编码方式。例如，可以决定将单声道数据（1通道）编码为立体声数据（2通道），并指定输出采样率：

In [5]:
desired_sample_rate = 32000
encoded_samples = encoder.to_tensor(format="wav", num_channels=2, sample_rate=desired_sample_rate)

stereo_samples_back = AudioDecoder(encoded_samples).get_all_samples()

print(stereo_samples_back)
play_audio(stereo_samples_back.data, rate=desired_sample_rate)

AudioSamples:
  data (shape): torch.Size([2, 96000])
  pts_seconds: 0.0
  duration_seconds: 3.0
  sample_rate: 32000

